In [ ]:
import pandas as pd
import numpy as np
import unidecode as ud


In [ ]:
import sqlite3
def connect_to_database(database_name):
    """
    Conectar a una base de datos SQLite
    
    Parameters
    ----------
    database_name : str
        Nombre de la base de datos
        
    Returns
    -------
    conn : sqlite3.Connection
        Conexión a la base de datos
        
    """
    conn = sqlite3.connect(database_name)
    return conn

def generate_query(table_name, columns=None, conditions=None):
    """
    Generar una query para buscar en la base de datos SQLite
    
    Parameters
    ----------
    table_name : str
        Nombre de la tabla en la base de datos
    columns : list
        Columnas a seleccionar
    conditions : list
        Condiciones para la búsqueda
        
    Returns
    -------
    query : str
        Query para buscar en la base de datos
        
    """
    
    query = f"SELECT {', '.join(columns) if columns else '*'} FROM {table_name}"
    if conditions:
        query += f" WHERE {conditions[0]}"
        for condition in conditions[1:]:
            query += f" AND {condition}"
    return query

def execute_query(conn, query, parameters=None):
    """

    Ejecutar una consulta en la base de datos SQLite
    
    Parameters
    ----------
    conn : sqlite3.Connection
        Conexión a la base de datos
    query : str
        Query a ejecutar
    parameters : tuple
        Parámetros a pasar a la consulta
    
    Returns
    -------
    results: list
        Lista de resultados de la consulta
        
    """
    cursor = conn.cursor()
    if parameters:
        cursor.execute(query, parameters)
    else:
        cursor.execute(query)
    results = cursor.fetchall()
    return results

def generate_conditions(nombres, edad=None, anio_eleccion=None):
    """
    Generar condiciones para la búsqueda de cédulas en la base de datos
    
    Parameters
    ----------
    nombres : str
        Nombres de la persona a buscar
    edad : float
        Edad de la persona a buscar
    anio_eleccion : int 
        Año de la elección
        
    Returns
    -------
    conditions : list
        Lista de condiciones para la búsqueda
    
    """
    values = nombres.split(" ")
    conditions = []
    for value in values:
        if len(value) > 2:
            conditions.append(f"NOMBRES LIKE '%{value}%'")
        else:
            conditions.append(f"NOMBRES LIKE '% {value}%'")
    if edad == 0.0:
        pass
    else:
        anio_nacimiento = anio_eleccion - edad
        # the anio_nacimiento could be a range of years
        conditions.append(f"SUBSTRING(FECH_NAC,7,4) BETWEEN '{anio_nacimiento - 1}' AND '{anio_nacimiento + 1}'")
    return conditions


def search_cedulas(nombre, edad, anio_eleccion, columns=["CEDULA","NOMBRES","FECH_NAC"]):
    """
    Buscar cédulas en la base de datos
    
    Parameters
    ----------
    nombre : str
        Nombres de la persona a buscar
    edad : float
        Edad de la persona a buscar
    anio_eleccion : int
        Año de la elección
    columns : list
        Columnas a seleccionar
        
    Returns
    -------
    result : list
        Lista de resultados de la búsqueda
    """
    database_name="cedulas.db"
    conn=connect_to_database(database_name)
    table_name="cedulas_data"
    condition=generate_conditions(nombre, edad, anio_eleccion)
    query=generate_query(table_name,columns, conditions=condition)
    print(query)
    result=execute_query(conn,query)

    for row in result:
        print(row)
    # print number of rows returned
    print(f"Number of rows returned: {len(result)}")
    conn.close()
    return result

In [ ]:
columns = ["CEDULA","NOMBRES","FECH_NAC"]
nombre_candidato="JURADO ZAMBRANO GENESIS GABRIELA"
edad=0.0
anio_eleccion=2006
search_cedulas(nombre_candidato,edad,anio_eleccion,columns)

In [ ]:
#read the presidetes.csv file
columns = ["CEDULA","NOMBRES","FECH_NAC"]

presidentes=pd.read_csv('presidentes.csv')
#Guardar los resultados en otro dataframe, ya que por cada presidente se puede tener mas de un resultado
resultados=pd.DataFrame(columns=["CEDULA","NOMBRES","FECH_NAC","CANDIDATO_NOMBRE","CANDIDATO_EDAD","ANIO"])
#list of dictionaries
info=[]
for index, row in presidentes.iterrows():
    nombre_candidato=row["CANDIDATO_NOMBRE"]
    edad=row["CANDIDATO_EDAD"]
    anio_eleccion=row["ANIO"]
    print(nombre_candidato,edad,anio_eleccion)
    results=search_cedulas(nombre_candidato,edad,anio_eleccion,columns)
    #Guardar los resultados en otro dataframe, ya que por cada presidente se puede tener mas de un resultado
    for result in results:
        cedula=result[0]
        nombre_=result[1]
        fecha_nacimiento=result[2]
        # add the results to the dataframe
        info.append({"CEDULA":cedula,"NOMBRES":nombre_,"FECH_NAC":fecha_nacimiento,"CANDIDATO_NOMBRE":nombre_candidato,"CANDIDATO_EDAD":edad,"ANIO":anio_eleccion})
resultados=pd.DataFrame(info)
resultados

In [ ]:
resultados.to_csv("resultados.csv",index=False)